# Salary Estimation - Data Cleaning

In [1]:
import pandas as pd

In [2]:
df=pd.read_csv("https://raw.githubusercontent.com/PlayingNumbers/ds_salary_proj/master/glassdoor_jobs.csv")

In [3]:
df.head()
df.columns

Index(['Unnamed: 0', 'Job Title', 'Salary Estimate', 'Job Description',
       'Rating', 'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors'],
      dtype='object')

In [4]:
df.columns
df.shape

(956, 15)

In [5]:
# Salary Estimate(Removing Glassdoor est.),Removing the 'K',splitting the salary for max,min,avg salaries

# Removing any -1 from the salary
df=df[df['Salary Estimate']!='-1']

# Removing the paranthesis by splitting the data
salary=df['Salary Estimate'].apply(lambda x: x.split('(')[0])

# Removing the 'K' and ''$'
minus_k=salary.apply(lambda x: x.replace('K','').replace('$',''))

# Removing employer provided salary and per hr
minus_hrps=minus_k.apply(lambda x: x.lower().replace('per hour','').replace('employer provided salary:',''))

# Getting max and min salary and also converting them into int type
df['min_sal']=minus_hrps.apply(lambda x: int(x.split('-')[0]))
df['max_sal']=minus_hrps.apply(lambda x: int(x.split('-')[1]))
df['avg_sal']=(df.min_sal+df.max_sal)/2

# Changing the column
df['Salary Estimate']=minus_hrps

In [6]:
# Job Description-Getting the skillset like python , r, excel etc.
df['python']=df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)
df['r studio']=df['Job Description'].apply(lambda x: 1 if 'r studio' in x.lower() else 0)
df['excel']=df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)
df['spark']=df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)


In [7]:
# Company Name- Removing the \n rating from the name
df['Company Name']=df['Company Name'].apply(lambda x: x.split('\n')[0])


In [8]:
# Location-splitting the city and the state
df['city']=df['Location'].apply(lambda x: x.split(',')[0])
df['state']=df['Location'].apply(lambda x: x.split(',')[1])
df.head()

,Unnamed: 0,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,...,Competitors,min_sal,max_sal,avg_sal,python,r studio,excel,spark,city,state
0,0,Data Scientist,53-91,"Data Scientist\nLocation: Albuquerque, NM\nEdu...",3.8,Tecolote Research,"Albuquerque, NM","Goleta, CA",501 to 1000 employees,1973,...,-1,53,91,72.0,1,0,1,0,Albuquerque,NM
1,1,Healthcare Data Scientist,63-112,What You Will Do:\n\nI. General Summary\n\nThe...,3.4,University of Maryland Medical System,"Linthicum, MD","Baltimore, MD",10000+ employees,1984,...,-1,63,112,87.5,1,0,0,0,Linthicum,MD
2,2,Data Scientist,80-90,"KnowBe4, Inc. is a high growth information sec...",4.8,KnowBe4,"Clearwater, FL","Clearwater, FL",501 to 1000 employees,2010,...,-1,80,90,85.0,1,0,1,1,Clearwater,FL
3,3,Data Scientist,56-97,*Organization and Job ID**\nJob ID: 310709\n\n...,3.8,PNNL,"Richland, WA","Richland, WA",1001 to 5000 employees,1965,...,"Oak Ridge National Laboratory, National Renewa...",56,97,76.5,1,0,0,0,Richland,WA
4,4,Data Scientist,86-143,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,"New York, NY","New York, NY",51 to 200 employees,1998,...,"Commerce Signals, Cardlytics, Yodlee",86,143,114.5,1,0,1,0,New York,NY


In [9]:
# HQ-Location-splitting the city 
df['hq_city']=df['Headquarters'].apply(lambda x: x.split(',')[0])

In [10]:
# Simplify Job Title and seniority

#Function to simplify the title
def title_simp(title):
    if 'data scientist' in title.lower():
        return 'data scientist'
    elif 'data engineer' in title.lower():
        return 'data engineer'
    elif 'analyst' in title.lower():
        return 'analyst'
    elif 'machine learning' in title.lower():
        return 'ml'
    elif 'manager' in title.lower():
        return 'manager'
    elif 'director' in title.lower():
        return 'director'
    else:
        return 'na'


# Function to simplify the seniority    
def seniority(title):
    if 'sr' in title.lower() or 'senior' in title.lower() or 'sr' in title.lower() or 'lead' in title.lower() or 'principal' in title.lower():
        return 'senior'
    elif 'jr' in title.lower() or 'jr.' in title.lower():
        return 'junior'
    else:
        return 'na'
    

# Run both the function
df['job_simp']=df['Job Title'].apply(title_simp)
df['job_title']=df['Job Title'].apply(seniority)

#df['job_title'].value_counts()
#df['job_simp'].value_counts()  

In [11]:
df.columns

Index(['Unnamed: 0', 'Job Title', 'Salary Estimate', 'Job Description',
       'Rating', 'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'min_sal', 'max_sal', 'avg_sal', 'python', 'r studio', 'excel', 'spark',
       'city', 'state', 'hq_city', 'job_simp', 'job_title'],
      dtype='object')

In [12]:
# Since there is a 'Los Angeles' in the list we need to fix that
df['state']=df['state'].apply(lambda x: x.replace('Los Angeles','CA'))

In [13]:
# Removing -1 from the Competitors and getting the number of competitors
df['num_comp']=df['Competitors'].apply(lambda x: len(x.split(',')) if x != '-1' else 0)

In [14]:
# Saving the file to csv
df.to_csv(r'SalaryData_cleaned.csv')